# NEO4j Python API

## 1. Setup

### 1.1. Import packages

In [ ]:
import neo4j as neo
import json

### 1.2. Define connection variables

In [ ]:
URL = 'bolt://localhost:7687'
AUTH = ('neo4j', 'neo4j_')

### 1.3. Test connection

In [ ]:
try:
    driver = neo.GraphDatabase.driver(URL, auth=AUTH, encrypted=False)
    print('* Connect successful.')
except:
    print('* Connect failed.')
finally:
    driver.close()

### 1.4. Support methods

#### 1.4.1. Query methods

In [ ]:
def do_query(tx, query, **args):
    result = tx.run(query, **args)
    return result

def run_transaction(statement_list):
    driver = neo.GraphDatabase.driver(URL, auth=AUTH, encrypted=False)
    try:
        with driver.session() as session:
            results = []
            for statement in statement_list:
                query = statement['statement']
                args = statement.get('args', {})
                method = statement.get('method', 'READ')

                if method == 'WRITE':
                    result = session.write_transaction(do_query, query, **args)
                else:
                    result = session.read_transaction(do_query, query, **args)
                
                results.append(result)
                
            return results
    finally:
        driver.close()

#### 1.4.2. Show results

In [ ]:
def pretty_output(results_lists):
    for n, results in enumerate(results_lists):
        results = results.values()
        for m, result in enumerate(results):
            for i, r in enumerate(result):
                print('* {}.{}.{}: {}'.format(n, m, i, r))

#### 1.4.3. Show neo4j node

In [ ]:
def show_node(node):
    print('* node information are: ')
    print('\tphysical id is: {}'.format(node.id))
    print('\tlabel is: {}'.format(next(iter(node.labels))))
    print('\tproperties are: ')
    print('\t\tkeys: {}'.format(list(node.keys())))
    print('\t\tvalues: {}'.format(list(node.values())))
    print('\t\tid: {}'.format(node['id']))
    print('\t\tname: {}'.format(node.get('name')))

#### 1.4.4. Show relationship

In [ ]:
def show_relationship(relationships):
    for r in relationships:
        r = r[0][0]
        print('-> relationship informations are:')
        print('\tid: {}'.format(r.id))
        print('\tlabel: {}'.format(r.type))
        print('\tstart node[id={}, name={}]'.format(r.start_node['id'], r.start_node['name']))
        print('\tend node[id={}, name={}]'.format(r.end_node['id'], r.end_node['name']))

## 2. Neo4j Statements

### 2.1. Object create and update

#### 2.1.1. Create or merge two persons named 'Alvin' and 'Emma'

In [ ]:
statements = [
    {
        'statement': 'MERGE (person:Person {id: $id, name: $name, gender: $gender}) '
                     'RETURN person',
        'args': {
            'id': 1,
            'name': 'Alvin',
            'gender': 'M'
        },
        'method': 'WRITE'
    },
    {
        'statement': 'MERGE (person:Person {id: $id, name: $name, gender: $gender}) '
                     'RETURN person',
        'args': {
            'id': 2,
            'name': 'Emma',
            'gender': 'F'
        },
        'method': 'WRITE'
    }
]

pretty_output(run_transaction(statements))

#### 2.1.2. Update person's birthdays

In [ ]:
statements = [
    {
        'statement': 'MATCH (person:Person)'
                     'WHERE person.id = $id '
                     'SET person.birthday = $birthday '
                     'RETURN person',
        'args': {
            'id': 1,
            'birthday': '1981-03-17'
        },
        'method': 'WRITE'
    },
    {
        'statement': 'MATCH (person:Person)'
                     'WHERE person.id = $id '
                     'SET person.birthday = $birthday '
                     'RETURN person',
        'args': {
            'id': 2,
            'birthday': '1985-03-29'
        },
        'method': 'WRITE'
    }
]

pretty_output(run_transaction(statements))

#### 2.1.3. Make relationship 'Married' between 'Alvin' and 'Emma'

In [ ]:
statements = [
    {
        'statement': 'MATCH (p1:Person), (p2:Person)'
                     'WHERE p1.id = $id1 AND p2.id = $id2 '
                     'MERGE (p1)-[r:Married]->(p2) '
                     'RETURN p1, p2, type(r)',
        'args': {
            'id1': 1,
            'id2': 2
        },
        'method': 'WRITE'
    }
]

pretty_output(run_transaction(statements))

#### 2.1.4. Create cat 'Lily' and make 'Feed' relationship to 'Alvin'

In [ ]:
statements = [
    {
        'statement': 'MATCH (person:Person) '
                     'WHERE person.id = $person_id '
                     'MERGE (cat:Cat { id: $cat_id, name: $cat_name }) '
                     'MERGE (person)-[r:Feed]->(cat) '
                     'RETURN person, cat, type(r)',
        'args': {
            'person_id': 1,
            'cat_id': 3,
            'cat_name': "Lily"
        },
        'method': 'WRITE'
    }
]

pretty_output(run_transaction(statements))

### 2.2. Query

#### 2.2.1. Find person who feed cat Lily 

In [ ]:
statements = [
    {
        'statement': 'MATCH (person:Person)-[r:Feed]->(cat:Cat) '
                     'WHERE cat.name = $cat_name '
                     'RETURN person, cat, type(r)',
        'args': {
            'cat_name': "Lily"
        }
    }
]

pretty_output(run_transaction(statements))

#### 2.2.2. Find persons who has relationship with cat 'Lily'

In [ ]:
statements = [
    {
        'statement': 'MATCH (person:Person)-[r*]->(cat:Cat) '
                     'WHERE cat.name = $cat_name '
                     'RETURN DISTINCT person.name',
        'args': {
            'cat_name': 'Lily'
        }
    }
]

pretty_output(run_transaction(statements))

#### 2.2.3. Find who has relationship with cat Lily

In [ ]:
statements = [
    {
        'statement': 'MATCH (p1:Person)-[:Married]-(p2:Person)-[r:Feed*]->(c:Cat) '
                     'WHERE c.name=$cat_name '
                     'RETURN DISTINCT p1.name',
        'args': {
            'cat_name': 'Lily'
        }
    }
]

pretty_output(run_transaction(statements))

#### 2.2.4. Find graph to cat Lily

In [ ]:
statements = [
    {
        'statement': 'MATCH g = (person:Person)-[r*]->(cat:Cat) '
                     'WHERE cat.name = $cat_name '
                     'RETURN g, length(g)',
        'args': {
            'cat_name': 'Lily'
        }
    }
]

pretty_output(run_transaction(statements))

#### 2.2.5. Find last children node in relationship chain

In [ ]:
statements = [
    {
        'statement': 'MATCH ()-[*]->(n:Cat) '
                     'WHERE NOT (n)-[]->() '
                     'RETURN DISTINCT n'
    }
]

pretty_output(run_transaction(statements))

#### 2.2.6. Match results

In [ ]:
statements = [
    {
        'statement': 'MATCH ()-[r*]->(n:Cat) '
                     'WHERE NOT (n)-[]->() '
                     'RETURN DISTINCT n'
    }
]

show_node(run_transaction(statements)[0].values()[0][0])

statements = [
    {
        'statement': 'MATCH (m)-[r*]->(n:Cat) '
                     'WHERE NOT (n)-[]->() '
                     'RETURN DISTINCT r, m, n'
    }
]

print()
show_relationship(run_transaction(statements)[0].values())

## 2.3. Delete nodes

### 2.3.1. Delete all relationships

In [ ]:
statements = [
    {
        'statement': 'MATCH ()-[r]->() '
                     'DELETE r '
                     'RETURN r'
    }
]

pretty_output(run_transaction(statements))

### 2.3.2. Delete all nodes

In [ ]:
statements = [
    {
        'statement': 'MATCH (n) '
                     'DELETE n '
                     'RETURN n'
    }
]

pretty_output(run_transaction(statements))

### 2.3.1. Delete by relationships

In [ ]:
statements = [
    {
        'statement': 'MATCH ()-[mr:Married]->() '
                     'MATCH ()-[fr:Feed]->() '
                     'OPTIONAL MATCH (ps:Person) '
                     'OPTIONAL MATCH(cs:Cat) '
                     'DELETE mr, fr, ps, cs '
                     'RETURN mr, fr, ps, cs'
    }
]

pretty_output(run_transaction(statements))

### 2.3.4. Query all nodes

In [ ]:
statements = [
    {
        'statement': 'MATCH (n)'
                     'RETURN n'
    }
]

pretty_output(run_transaction(statements))

## 2.4. Misc

### 2.4.1. Import

#### 2.4.1.1. Imprt from csv

In [ ]:
statements = [
    {
        'statement': 'LOAD CSV WITH HEADERS FROM "file:///data.csv" AS line '
                     'MERGE (p1:Person {id: line["id"], name: line["name"]}) '
                     'MERGE (p2:Person {id: line["married"]}) '
                     'MERGE (p1)-[:Married]->(p2) '
                     'RETURN p1, p2'
    }
]

pretty_output(run_transaction(statements))

#### 2.4.1.2. Query import result

In [ ]:
statements = [
    {
        'statement': 'MATCH (n)-[r]->(m)'
                     'RETURN n, r, m'
    }
]

pretty_output(run_transaction(statements))

### 2.4.2. Fulltext index

#### 2.4.2.1. List all fulltext analyzers

In [ ]:
statements = [
    {
        'statement': 'CALL db.index.fulltext.listAvailableAnalyzers()'
    }
]

pretty_output(run_transaction(statements))

#### 2.4.2.2. Create fulltext index

In [ ]:
statements = [
    {
        'statement': 'CALL db.index.fulltext.createNodeIndex("ix_name", ["Person", "Cat"], ["name"], {analyzer: "cjk"})'
    }
]

pretty_output(run_transaction(statements))
print('* index created')

#### 2.4.2.3. Query by fulltext index

In [ ]:
statements = [
    {
        'statement': 'CALL db.index.fulltext.queryNodes("ix_name", "Alvin") YIELD node, score '
                     'RETURN node, score'
    }
]

pretty_output(run_transaction(statements))

#### 2.4.2.4. Query by fulltext index

In [ ]:
statements = [
    {
        'statement': 'CALL db.index.fulltext.drop("ix_name") '
    }
]

pretty_output(run_transaction(statements))